In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/u

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-11 03:05:10--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.1s    

2022-04-11 03:05:10 (6.01 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonApparelCloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

amazon_apparel_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), inferSchema=True, sep='\t', timestampFormat="mm/dd/yy")
amazon_apparel_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [6]:
amazon_apparel_df = amazon_apparel_df.dropna()
amazon_apparel_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [10]:
reviews_df = amazon_apparel_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
reviews_df.show(10)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|            0|          0|   N|                Y|
|          5|            1|          2|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          3|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [12]:
reviews_df = reviews_df.dropna(how='any')
reviews_df.dropDuplicates()
reviews_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|            0|          0|   N|                Y|
|          5|            1|          2|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          3|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|           

In [13]:
vine_df = reviews_df.filter("total_votes>=10").filter(reviews_df["helpful_votes"]/reviews_df["total_votes"] >= 0.5)
vine_df.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           12|         12|   N|                Y|
|          5|            7|         12|   N|                N|
|          5|           13|         13|   N|                N|
|          5|            8|         10|   N|                N|
|          5|           15|         18|   N|                N|
|          5|           30|         32|   N|                N|
|          5|           51|         52|   N|                N|
|          5|           29|         36|   N|                Y|
|          5|          148|        150|   N|                Y|
|          4|           23|         26|   N|                N|
|          4|           11|         13|   N|                N|
|          5|           16|         16|   N|                Y|
|          5|            8|         12|   N|           

In [19]:
from pyspark.sql.functions import col, avg
paid_df = vine_df.filter("vine='Y'")
not_paid_df = vine_df.filter("vine='N'")

In [20]:
paid_df.describe().show()

+-------+------------------+------------------+------------------+----+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|vine|verified_purchase|
+-------+------------------+------------------+------------------+----+-----------------+
|  count|                74|                74|                74|  74|               74|
|   mean| 4.013513513513513|27.635135135135137|30.013513513513512|null|             null|
| stddev|0.9861123785363181|  36.3097686106209| 37.85462214344639|null|             null|
|    min|                 1|                 7|                10|   Y|                N|
|    max|                 5|               240|               248|   Y|                Y|
+-------+------------------+------------------+------------------+----+-----------------+



In [21]:
not_paid_df.describe().show()

+-------+------------------+------------------+------------------+------+-----------------+
|summary|       star_rating|     helpful_votes|       total_votes|  vine|verified_purchase|
+-------+------------------+------------------+------------------+------+-----------------+
|  count|            110227|            110227|            110227|110227|           110227|
|   mean|3.8219764667458973|27.713545682999627|29.925444763987045|  null|             null|
| stddev| 1.472264104539861|148.72325547461637|152.17689529619136|  null|             null|
|    min|                 1|                 5|                10|     N|                N|
|    max|                 5|             41278|             41889|     N|                Y|
+-------+------------------+------------------+------------------+------+-----------------+



In [17]:
five_star = paid_df[paid_df['star_rating'] == 5].count()
print(five_star)
all_paid = paid_df.count()
print(all_paid)

28
74


In [18]:
print(five_star/all_paid)

0.3783783783783784


In [24]:
not_paid_five_star = not_paid_df[not_paid_df['star_rating'] == 5].count()
print(not_paid_five_star)
all_unpaid = not_paid_df.count()
print(all_unpaid)

55701
110227


In [26]:
print(not_paid_five_star/all_unpaid)

0.5053299100946229


## **Analisys:**

The average rating from non-Vine customers is 3.8 with std deviation of 0.98, and this is much higher than the 1.4 from Vine customers.

Vine customers tend to give higher ratings and pretty focusing on the higher ratings.

The percentage of 5-star reviews in Vine to non-Vine reviews (37% to 50.5%).

Vine customer might be bias towards the apparel products. 
